In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-07-22"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
6841,2024-07-22,Porto Rico Bsn,21:00,Capitanes De Arecibo,Osos de Manati,1.65,2.15,195.5,1.83,1.83,0.0,0.0,0.0,hGvH4XSg,0.606061,0.465116,0.546448,0.546448,0.071177,164.746,30.806799,0.186996,1.2,1.643168,1.369306,203.58,1.818,0.383171,0.210765,-75.0,208.602,71.263712,0.341625,1.8,1.643168,0.912871,179.80,1.956,0.603680,0.308630,50.0,87,116,2.34,1.55,155.738,238.838,0.186081,0.000000,NaN,0.97,0.194,3.350515,0.707194,0.8,0.092806,-2.94,-0.588,-1.955782,0.391323,0.3,-0.091323
6842,2024-07-22,Eua Nba,21:00,Miami Heat,Memphis Grizzlies,2.09,1.69,185.5,1.83,1.83,0.0,0.0,0.0,tb0Km44M,0.478469,0.591716,0.546448,0.546448,0.070185,161.042,27.549518,0.171070,3.0,0.000000,0.000000,161.16,1.530,0.135093,0.088296,86.0,152.420,26.493735,0.173821,3.0,0.000000,0.000000,140.58,1.510,0.175784,0.116413,58.0,102,99,1.58,1.42,253.466,310.660,0.149652,0.000000,NaN,-0.56,-0.112,-9.732143,0.596868,0.6,0.003132,7.68,1.536,0.449219,0.350987,0.5,0.149013
6843,2024-07-22,Mundo Amistoso Internacional,15:00,Sérvia,Grécia,1.51,2.45,166.5,1.83,1.87,0.0,0.0,0.0,I3oL388F,0.662252,0.408163,0.546448,0.534759,0.070415,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,119,67,1.03,1.26,0.000,0.000,0.335697,0.015289,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6844,2024-07-22,Mundo Amistoso Internacional,16:00,EUA,Alemanha,1.05,8.15,180.5,1.83,1.87,0.0,0.0,0.0,E9RgGjzf,0.952381,0.122699,0.546448,0.534759,0.075080,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,105,104,1.08,1.03,0.000,0.000,1.091404,0.015289,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6845,2024-07-22,Mundo Amistosos Internacionais,15:00,Espanha F,Austrália F,3.12,1.33,139.5,1.85,1.85,0.0,0.0,0.0,d2zUITmB,0.320513,0.751880,0.540541,0.540541,0.072393,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,59,85,1.03,2.65,0.000,0.000,0.568863,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6846,2024-07-22,Porto Rico Bsn,21:00,Leones De Ponce,Mets de Guaynabo,1.63,2.19,176.5,1.80,1.90,0.0,0.0,0.0,821s8iEL,0.613497,0.456621,0.555556,0.526316,0.070118,204.384,49.114859,0.240307,1.8,1.643168,0.912871,235.71,2.326,0.636734,0.273746,13.0,145.392,28.186846,0.193868,1.8,1.643168,0.912871,113.71,1.650,0.295719,0.179224,-10.0,81,83,2.91,1.37,172.938,167.348,0.207319,0.038222,NaN,4.09,0.818,0.770171,0.561438,0.8,0.238562,0.63,0.126,9.444444,0.488356,0.6,0.111644


## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

Sem jogos
